In [1]:
import librosa #Paquete de analisis de audio
import librosa.display
import IPython.display as ipd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
from PIL import Image
import pathlib
import csv

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import preprocessing

In [2]:
data = pd.read_csv('/Users/andre/OneDrive/Escritorio/Biometry_Voice/BiometriaPorVoz-master/data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,Audio1.wav,0.374350,0.053845,1965.682320,1615.392815,3758.319355,0.115206,-251.557526,122.310768,-40.907310,...,-13.192103,0.880217,-18.683870,3.074591,-13.036599,-10.126388,-4.930336,-7.613022,-9.723351,['id10001']
1,Audio10.wav,0.343509,0.055478,1771.943045,1531.637893,3364.967140,0.096564,-258.922302,130.249344,-46.288002,...,-11.412473,2.987354,-18.023815,1.100359,-9.930485,-12.599640,-5.048801,-5.932458,-8.263415,['id10001']
2,Audio11.wav,0.361852,0.037427,1845.683655,1720.180568,3548.118882,0.106438,-293.797058,125.556908,-31.383408,...,-5.537814,3.552005,-18.380121,7.791273,-14.333706,-0.354537,-5.532031,-1.329604,-3.782263,['id10001']
3,Audio12.wav,0.370827,0.058004,1630.207162,1549.959824,3234.462260,0.081284,-272.567719,135.305984,-27.281111,...,-9.818530,1.679797,-13.206263,5.363429,-8.567389,-5.418140,-3.591171,-1.071382,-5.491786,['id10001']
4,Audio13.wav,0.386647,0.030166,1629.170753,1570.610481,3089.857308,0.096611,-344.663879,130.001419,-9.767311,...,-4.316083,3.136457,-7.597161,1.629692,-8.392146,-3.011672,-4.094692,-3.789023,-3.145382,['id10001']


In [3]:
data.shape

(161, 28)

In [4]:
data = data.drop(['filename'],axis=1)

In [5]:
idList = data.iloc[:, -1] #Con "iloc" Seleccionamos la columna de los id's
encoder = LabelEncoder()
y = encoder.fit_transform(idList) #Ajustar a los datos, luego transformarlo.

In [6]:
scaler = StandardScaler() #Estandariza las características eliminando la media y escalando a la varianza unitaria
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float)) #Ajusta el transformador a X y devuelve una versión transformada de X.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #Dividir los datos en el conjunto de entrenamiento y pruebas

In [8]:
len(y_train)

128

In [9]:
len(y_test)

33

In [10]:
X_train[10] 

array([ 0.27478781,  0.06338621,  2.10441177,  1.24510978,  1.87537361,
        1.98217791,  0.90611507, -1.90250213, -1.11090513,  1.23459213,
       -1.84663373, -0.2217592 , -1.7736343 , -1.27380488, -0.53352751,
       -1.16438892, -1.90795991, -1.58881625, -0.78824254, -1.80526537,
       -0.52255423, -0.85743264, -1.72199883, -0.92533933, -1.88350986,
       -2.21055066])

In [11]:
#Construyendo la red neuronal
from keras import models
from keras import layers

model = models.Sequential() #iniciamos los métodos de modelo seqüenciales
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(32, activation='softmax'))

Using TensorFlow backend.


In [12]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
history = model.fit(X_train,
                    y_train,
                    epochs=30,
                    batch_size=256)


Epoch 1/30
128/128 [==============================] - 1s 10ms/step - loss: 3.3333 - accuracy: 0.0703
Epoch 2/30
128/128 [==============================] - 0s 122us/step - loss: 3.0782 - accuracy: 0.5078
Epoch 3/30
128/128 [==============================] - 0s 244us/step - loss: 2.8238 - accuracy: 0.7422
Epoch 4/30
128/128 [==============================] - 0s 0us/step - loss: 2.5638 - accuracy: 0.9375
Epoch 5/30
128/128 [==============================] - 0s 0us/step - loss: 2.2975 - accuracy: 0.9609
Epoch 6/30
128/128 [==============================] - 0s 244us/step - loss: 2.0214 - accuracy: 0.9609
Epoch 7/30
128/128 [==============================] - 0s 0us/step - loss: 1.7362 - accuracy: 0.9766
Epoch 8/30
128/128 [==============================] - 0s 0us/step - loss: 1.4475 - accuracy: 0.9766
Epoch 9/30
128/128 [==============================] - 0s 0us/step - loss: 1.1638 - accuracy: 0.9766
Epoch 10/30
128/128 [==============================] - 0s 122us/step - loss: 0.8975 - accura

In [14]:
test_loss, test_acc = model.evaluate(X_test,y_test)

33/33 [==============================] - 0s 7ms/step


In [15]:
print('test_acc: ',test_acc)

test_acc:  0.9696969985961914


In [16]:
x_val = X_train[:558]
partial_x_train = X_train[558:]

y_val = y_train[:558]
partial_y_train = y_train[558:]

In [17]:
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 0 samples, validate on 128 samples
Epoch 1/30


UnboundLocalError: local variable 'batch_index' referenced before assignment

In [ ]:
results

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions[0].shape

In [ ]:
np.sum(predictions[0])

In [ ]:
np.argmax(predictions[0])